In [1]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test DFA-2-RegExp

In [2]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, Tuple } from "recursive-set";
import {
  State,
  Char,
  DFA,
  RegExp,
  TransRelDet,
  key,
  dfa2regexp,
  BinaryOp,
  UnaryOp
} from "./05-DFA-2-RegExp";

// Hinzugefügt: Import von dfa2dot
import { dfa2dot } from "./FSM-2-Dot";

In [3]:
// Singletons {1}, {2}, {3}
const S0 = new RecursiveSet(1);
const S1 = new RecursiveSet(2);
const S2 = new RecursiveSet(3);

// Q = { {1}, {2}, {3} }
const Q = new RecursiveSet<RecursiveSet<number>>(S0, S1, S2);

// Σ = {a, b}
const Sigma = new RecursiveSet<string>("a", "b");

// δ (Übergangsfunktion)
const delta: TransRelDet = new Map<string, RecursiveSet<number>>();
delta.set(key(S0, "a"), S1); // (1,a)->2
delta.set(key(S1, "b"), S2); // (2,b)->3
delta.set(key(S2, "a"), S1); // (3,a)->2

// Startzustand und akzeptierende Zustände
const q0 = S0;
const A = new RecursiveSet<RecursiveSet<number>>(S2);

// DFA Objekt
const dfa: DFA = { Q, Sigma, delta, q0, A };

In [4]:
const { dot } = dfa2dot(dfa);
const viz = await instance();
display.html(viz.renderString(dot, { format: "svg" }));

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.1 (20251006.0113)
 -->
<!-- Title: Deterministic FSM Pages: 1 -->
 
 
 Deterministic FSM 
 
<!-- 1 -->
 
 1 
 
 
<!-- S0 -->
 
 S0 
 
 S0 
 
<!-- 1->S0 -->
 
 1->S0 
 
 
 
<!-- S1 -->
 
 S1 
 
 S1 
 
<!-- S0->S1 -->
 
 S0->S1 
 
 
 a 
 
<!-- S2 -->
 
 S2 
 
 
 S2 
 
<!-- S1->S2 -->
 
 S1->S2 
 
 
 b 
 
<!-- S2->S1 -->
 
 S2->S1 
 
 
 a

In [5]:
function prettyPrintRegExp(r: RegExp): string {
  if (typeof r === 'number' || typeof r === 'string') {
    return typeof r === 'string' ? `'${r}'` : String(r);
  }
  
  if (r instanceof Tuple) {
    // Wir nutzen den Iterator des Tuples
    const elements: string[] = [];
    for (const el of r) {
        // Rekursiver Aufruf für jedes Element (muss als RegExp gecastet werden, 
        // da Tuple auch Operatoren enthalten kann, die Strings sind)
        // Wir prüfen einfachheitshalber, ob es ein RegExp ist oder ein Operator-String
        elements.push(prettyPrintRegExp(el as RegExp));
    }
    return `(${elements.join(", ")})`;
  }
  
  return String(r);
}

// Berechnung des regulären Ausdrucks aus dem DFA
const r: RegExp = dfa2regexp(dfa);

console.log("Unvereinfachter regulärer Ausdruck:");
console.log(prettyPrintRegExp(r));

Unvereinfachter regulärer Ausdruck:
(((0, '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', ('ε', '+', 0))), '+', ((('a', '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '⋅', ((('ε', '+', 0), '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '*')), '⋅', ('b', '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', ('ε', '+', 0))))), '+', ((((('ε', '+', 0), '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', 0)), '+', ((('a', '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '⋅', ((('ε', '+', 0), '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '*')), '⋅', (0, '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', 0)))), '⋅', (((('ε', '+', 0), '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', 0)), '+', ((('a', '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '⋅', ((('ε', '+', 0), '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', 'a')), '*')), '⋅', (0, '+', (('b', '⋅', (('ε', '+', 0), '*')), '⋅', 0)))), '*')), '⋅', ((0, '+', ((0, '⋅', (('ε', '+', 0), '*')), '⋅', ('ε', '+', 0))), '+', ((('a', '+', ((0, '⋅', (('ε', '+', 0), '*'))

## Helpfunctions:

In [6]:
function isZero(r: RegExp): boolean {
  return r === 0;
}

function isEps(r: RegExp): boolean {
  return r === "ε" || r === "𝜀";
}

function eq(a: RegExp, b: RegExp): boolean {
  // JSON.stringify funktioniert auch bei Tuple (es serialisiert die 'values')
  return JSON.stringify(a) === JSON.stringify(b);
}

As this regular expression is nearly unreadable,  The notebook `Rewrite.ipynb` contains the definition of the function `simplify` that can be used to simplify this expression.

In [7]:
function simplify(r: RegExp): RegExp {
  if (typeof r === "string" || typeof r === "number") return r;
  if (!(r instanceof Tuple)) return r;

  // Wir holen uns die Länge und Elemente über die Tuple-API
  const len = r.length;

  // ---------- Kleene Star ----------
  // Struktur: [RegExp, "*"]
  if (len === 2 && r.get(1) === "*") {
    const innerRaw = r.get(0) as RegExp;
    let inner = simplify(innerRaw);

    // 0* → ε
    if (isZero(inner)) return "ε"; 

    // ε* → ε
    if (isEps(inner)) return "ε"; 

    // (r*)* → r*
    // Prüfung: ist inner auch ein Tuple der Länge 2 mit '*'?
    if (inner instanceof Tuple && inner.length === 2 && inner.get(1) === "*") {
      return inner;
    }

    // (ε + r)* → r*
    // Prüfung: inner ist Tuple [ε, +, r] oder [r, +, ε] ?
    // Hier prüfen wir speziell: [ε, '+', r]
    if (
      inner instanceof Tuple &&
      inner.length === 3 &&
      inner.get(1) === "+" &&
      (isEps(inner.get(0) as RegExp) || isZero(inner.get(0) as RegExp))
    ) {
      const r2 = simplify(inner.get(2) as RegExp);
      // Rückgabe als Tuple
      return new Tuple(r2, "*" as UnaryOp);
    }

    return new Tuple(inner, "*" as UnaryOp);
  }

  // ---------- Binäre Operationen (+ oder ⋅) ----------
  // Struktur: [RegExp, Op, RegExp]
  if (len === 3) {
    const left = simplify(r.get(0) as RegExp);
    const op = r.get(1) as BinaryOp;
    const right = simplify(r.get(2) as RegExp);

    // ------------------ +
    if (op === "+") {
      if (isZero(left)) return right;
      if (isZero(right)) return left;
      if (eq(left, right)) return left;
      return new Tuple(left, "+" as BinaryOp, right);
    }

    // ------------------ ⋅
    if (op === "⋅") {
      if (isZero(left) || isZero(right)) return 0;
      if (isEps(left)) return right;
      if (isEps(right)) return left;
      return new Tuple(left, "⋅" as BinaryOp, right);
    }
  }

  return r;
}

10:22 - This expression is not callable.
10:22 - Each member of the union type '(<K extends keyof [RegExp, "*"]>(index: K) => [RegExp, "*"][K]) | (<K extends keyof [RegExp, BinaryOp, RegExp]>(index: K) => [RegExp, BinaryOp, RegExp][K])' has signatures, but none of those signatures are compatible with each other.
11:24 - This expression is not callable.
11:24 - Each member of the union type '(<K extends keyof [RegExp, "*"]>(index: K) => [RegExp, "*"][K]) | (<K extends keyof [RegExp, BinaryOp, RegExp]>(index: K) => [RegExp, BinaryOp, RegExp][K])' has signatures, but none of those signatures are compatible with each other.
22:63 - This expression is not callable.
22:63 - Each member of the union type '(<K extends keyof [RegExp, "*"]>(index: K) => [RegExp, "*"][K]) | (<K extends keyof [RegExp, BinaryOp, RegExp]>(index: K) => [RegExp, BinaryOp, RegExp][K])' has signatures, but none of those signatures are compatible with each other.
32:13 - This expression is not callable.
32:13 - Each memb

In [ ]:
let s = simplify(r);
s = simplify(s); // Zweiter Durchlauf für verschachtelte Vereinfachungen

console.log("Vereinfachter Ausdruck (Struktur):");
console.log(prettyPrintRegExp(s));

The function `regexp_2_string` takes a regular expression that is represented as a nested tuple and transforms it into a string.

In [ ]:
function regexpToString(r: RegExp): string {
  // leere Sprache
  if (r === 0) return "∅";

  // epsilon
  if (r === "ε" || r === "𝜀") return "ε";

  // einzelnes Symbol
  if (typeof r === "string") return r;

  // Tuple-Fälle
  if (r instanceof Tuple) {
    const len = r.length;

    // -----------------------------------------
    // Binäre Operatoren: [r1, '⋅', r2] / [r1, '+', r2]
    // -----------------------------------------
    if (len === 3) {
      const r1 = r.get(0) as RegExp;
      const op = r.get(1) as string;
      const r2 = r.get(2) as RegExp;

      if (op === "⋅") {
        const s1 = regexpToString(r1);
        const s2 = regexpToString(r2);
        
        // Einfache Heuristik für Klammern:
        // Wenn r1 oder r2 eine Summe (+) ist, klammern.
        const p1 = (r1 instanceof Tuple && r1.get(1) === '+') ? `(${s1})` : s1;
        const p2 = (r2 instanceof Tuple && r2.get(1) === '+') ? `(${s2})` : s2;
        
        return p1 + p2; 
      }

      if (op === "+") {
        return regexpToString(r1) + "+" + regexpToString(r2);
      }
    }

    // -----------------------------------------
    // Kleene-Stern: [expr, '*']
    // -----------------------------------------
    if (len === 2 && r.get(1) === "*") {
      const inner = r.get(0) as RegExp;
      const sInner = regexpToString(inner);

      // Symbol* oder (Ausdruck)*
      if (typeof inner === "string" || typeof inner === "number") {
        return sInner + "*";
      }

      return "(" + sInner + ")*";
    }
  }

  return JSON.stringify(r);
}

In [ ]:
console.log("\nEndgültiger regulärer Ausdruck (String):");
console.log(regexpToString(s));